### **Mount Google Drive/Parent Directory**

In [67]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### **Install and Import Libraries**

In [68]:
# Library to read .nii Images
!pip install nibabel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import cv2
import copy
import numpy as np
import numpy as np                                            
import matplotlib.pyplot as plt                 
from scipy.stats import multivariate_normal
import random  
import time  
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
# Library to read .nii Images
import nibabel as nib

**Image Slice Visualization**

In [70]:
# Visualize 2D Slice from 3D Image
def slice_show(image, slice_no, title):
    """
    Inputs: 
    image => Nifti Image that need to be visualized,
    slice_no => Slice Number from 1 to 48
    title => The title of the Image
    
    output: Plot Image.

    """ 
    plt.figure()
    plt.axis('off')
    plt.title(title)
    plt.imshow(image[:,:,slice_no].T, cmap='gray')

### **Evaluation Metric**

In [71]:
def tissue_visualization(Seg_CSF, GT_CSF, Seg_GM, GT_GM, Seg_WM, GT_WM, number_of_slice):

  """
  Inputs: 
  Seg_CSF => CSF class data of the brain tissue after segmentation,
  GT_CSF => Ground truth of CSF Class,
  Seg_GM => GM class data of the brain tissue after segmentation,
  GT_GM => Ground truth of GM Class,
  Seg_WM => WM class data of the brain tissue after segmentation,
  GT_WM => Ground truth of WM Class,
  
  output: Plot Images.

  """ 

  #Visualize each tissue separately
  fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots( 2, 3, figsize=(10,6))

  ax1.set_title("Class CSF #{}".format(number_of_slice))
  img1 = ax1.imshow(np.transpose(Seg_CSF[:,:,number_of_slice]), cmap = "gray")
  ax2.set_title("Class GM #{}".format(number_of_slice))
  img2 = ax2.imshow(np.transpose(Seg_GM[:,:,number_of_slice]), cmap = "gray")
  ax3.set_title("Class WM #{}".format(number_of_slice))
  img3 = ax3.imshow(np.transpose(Seg_WM[:,:,30]), cmap = "gray")
  ax4.set_title("Ground Truth CSF #{}".format(number_of_slice))
  img4 = ax4.imshow(np.transpose(GT_CSF[:,:,number_of_slice]), cmap = "gray")
  ax5.set_title("Ground Truth of GM #{}".format(number_of_slice))
  img5 = ax5.imshow(np.transpose(GT_GM[:,:,number_of_slice]), cmap = "gray")
  ax6.set_title("Ground Truth of WM #{}".format(number_of_slice))
  img6 = ax6.imshow(np.transpose(GT_WM[:,:,number_of_slice]), cmap = "gray")
  

  ax1.axis('off')
  ax2.axis('off')
  ax3.axis('off')
  ax4.axis('off')
  ax5.axis('off')
  ax6.axis('off')
  plt.tight_layout()
  plt.show()

In [72]:
def diceScoreSimilarity(segmented_image_nifi, ground_truth_nifti):
  
    """
    Inputs: 
    segmented_image_nifi => Segmeneted image in nii format,
    ground_truth_nifti => ground truth in nifti format,

    output: Dice Score for all the three classes separately.

    """ 
           
    # Compute DICE
    def dice(SI, GT):
        #   2 * TP / (FN + (2 * TP) + FP)
        intersection = np.logical_and(SI, GT)
        return 2. * intersection.sum() / (SI.sum() + GT.sum())
    
    # Dice  for CSF
    Seg_CSF = (segmented_image_nifi == 1)
    GT_CSF = (ground_truth_nifti == 1)
    dice_CSF = dice(Seg_CSF, GT_CSF)
    # Dice  for GM
    Seg_GM = (segmented_image_nifi == 2)
    GT_GM = (ground_truth_nifti == 2)
    dice_GM = dice(Seg_GM, GT_GM)
    # Dice  for WM
    Seg_WM = (segmented_image_nifi == 3)
    GT_WM = (ground_truth_nifti == 3)
    dice_WM = dice(Seg_WM, GT_WM)

    #  Visualize each tissue class separately
    number_of_slice = 24
    tissue_visualization(Seg_CSF, GT_CSF, Seg_GM, GT_GM, Seg_WM, GT_WM, number_of_slice)

    return dice_CSF, dice_GM, dice_WM

### **K-means Clustering**

In [73]:
def KmeansInitialization(brain_data, num_clusters):

    """
    Inputs: 
    brain_data => extracted brain_data excluding the black background,
    num_clusters => number of classes for segmentation ( here 3 )
    
    output: 
    mean_generation => mean initialized after k-means (size: N*M where N is the number of the classes and M is the number of modalities used)
    covariance_computation => covariance computed after the data segmentation is done on the data, 
    (covariance for multimodal data, variance for single modal data) (size: N*M*M where N is the number of the classes and M is the number of modalities used)
    a_vector => prior probabilities of each class ( size: N * 1 where N is the number of the classes )
    Kmeans_predict_updated => labels assigned by Kmeans to each image pixels (X*1 where is X is number of the pixels in the image)

    """ 

    """
    Kmeans Clustering is used from sklearn.cluster.KMeans
    Input: n_clusters= Number of cluster
          K-means++: initial cluster centers for k-mean clustering in a smart way to speed up convergence
          Random State :Determines random number generation for centroid initialization. 
                        Use an int to make the randomness deterministic
    Output:
          Kmeans_predict= level index
          Centroid= Mean
    """ 
    # K-means Clustering
    kmeans=KMeans(n_clusters = num_clusters,  init = 'k-means++', random_state = 0).fit(brain_data)
    Kmeans_predict = kmeans.predict(brain_data)
    centroids = kmeans.cluster_centers_

    # Sorting centroids using T1 weighted mean
    centroids_t1 = centroids[:,0]
    min_idx = np.argmin(centroids_t1, axis=0)
    max_idx = np.argmax(centroids_t1, axis=0)

    centroids_updated = np.zeros(centroids.shape)
    Kmeans_predict_updated = np.zeros(Kmeans_predict.shape)

    # Updating centroid after sorting according to mean values
    centroids_updated[0] = centroids[min_idx]
    centroids_updated[2] = centroids[max_idx]

    for i in [0,1,2]:
      if (min_idx != i and max_idx != i):
        mid_idx = i
        centroids_updated[1] = centroids[i]

    # Making the levels according to the ground truth (not starting from zero)
    Kmeans_predict_updated[Kmeans_predict==min_idx] = 1
    Kmeans_predict_updated[Kmeans_predict==mid_idx] = 2 
    Kmeans_predict_updated[Kmeans_predict==max_idx] = 3

    # Assigning classes to tissues
    CSF_tissue = brain_data[Kmeans_predict_updated == 1]
    GM_tissue = brain_data[Kmeans_predict_updated == 2]
    WM_tissue = brain_data[Kmeans_predict_updated == 3]

    # Computing mean and covariance
    mean_CSF = np.mean(CSF_tissue, axis = 0)
    mean_GM = np.mean(GM_tissue, axis = 0)
    mean_WM = np.mean(WM_tissue , axis = 0)
    cov_CSF = np.cov(CSF_tissue, rowvar = False)
    cov_GM = np.cov(GM_tissue, rowvar = False)
    cov_WM = np.cov(WM_tissue , rowvar = False)

    mean_generation = np.vstack((mean_CSF, mean_GM, mean_WM))
    covariance_computation = np.stack((cov_CSF, cov_GM, cov_WM))

    # Computing prior probabilities of each class
    pp_CSF = CSF_tissue.shape[0]/brain_data.shape[0]
    pp_GM = GM_tissue.shape[0]/brain_data.shape[0]
    pp_WM = WM_tissue.shape[0]/brain_data.shape[0]

    a_vector = np.vstack((pp_CSF, pp_GM, pp_WM))

    return mean_generation, covariance_computation, a_vector, Kmeans_predict_updated

### **EM Algorithm**

**Initialization**

In [108]:
def parameters_initialization(clusters, choice, meta_data_dice, no_modalities ):

  """
  Inputs: 
  clusters : number of clusters (here 3)
  choice: choice of initilization, 1 for Kmeans, 0 for Random,
  meta_data_dice: Dictionary containing all the brain image data to be used in EM

  output: mean, covariance and priors after initialization either by Kmeans or Random.

  """ 
  #Extracting the brain image data to be used for the algorithm
  x = meta_data_dice['brain_data']
  brain_indices = meta_data_dice['brain_indices']
  t1_data = meta_data_dice['t1_data']
  feature_data = meta_data_dice['feature_data']
  brain_image_nifti = meta_data_dice['brain_image_nifti']
  ground_truth = meta_data_dice['ground_truth']
  
  if choice == 1:

    # K-means Clustering Initialization
    mean_generation, covariance_computation, a_vector, Kmeans_predict_updated = KmeansInitialization(x, clusters)

    # print(covariance_computation.shape)

    # Clusteres Image Generation By Kmeans
    clustered_data =  np.zeros(feature_data.shape[0])
    clustered_data[brain_indices] = Kmeans_predict_updated
    clustered_img = np.reshape(clustered_data, brain_image_nifti.shape)

    # Saving the images in Nifti Format
    brain_seg_data = nib.Nifti1Image(clustered_img, t1_data.affine, t1_data.header)
    brain_seg_nifti = brain_seg_data.get_fdata()

    # Image Visualization after Kmeans Clustering
    titlek = 'Image after Kmeans'
    slice_show(brain_seg_nifti, 24, titlek)

    # Evaluation using DICE Score
    print(" After applying K-means :")
    dice_CSF, dice_GM, dice_WM = diceScoreSimilarity(brain_seg_nifti, ground_truth)
    print("CSF DICE = {}".format(dice_CSF), "GM DICE = {}".format(dice_GM), "WM DICE = {}".format(dice_WM))
    print('\n')

  elif choice == 2:
    mean_generation = np.zeros((clusters, no_modalities))  # initializing the mean 
    covariance_computation = np.zeros((clusters, no_modalities, no_modalities))  # initializing the cov
    for k in range(clusters):
      r = random.sample(range(0, x.shape[0]), clusters)  # getting random coordinates
      mean_generation[k] = x[r[k]]  # getting random data points as the mean
      temp = np.random.normal(0, 1000, size=(no_modalities, no_modalities))
      covariance_computation[k] = np.dot(temp.transpose(), temp)  # computing the covariance matrix
    print(covariance_computation)  
    #Initialization of the mixture weights
    a_values= 1/clusters
    a_vector= np.full(clusters, a_values)

    #Reshaping of the mixture weights vector to make it a column vector

    a_vector= a_vector.reshape((clusters,-1))

      
  parameters= {'mean': mean_generation, 'covariance': covariance_computation,
              'mixture_weights': a_vector}

  #The function returns the generated mean, covariance computed with respect to
  #the image data and the mixture weights vector. 
  #All parameters are returned 
  return parameters

**Expectation**

In [75]:
def ExpectationStep(gmm):

  """
  Inputs: 
  gmm => matrix containing the gaussian mixture models (probability density functions).
  The size of this matrix is (N,clusters), where N corresponds to the number of datapoints for each
  one of the modalities
  and K corresponds to the number of clusters (tissues to be segmented)
  
  output: weights_vector=> Updated membership weights for all data points 

  """ 

  #Parameter gmm: Mixture model for the current datapoint
  numerator= gmm 
  #Denominator of the equation
  denominator= np.sum(gmm, axis=1)
  denominator= denominator.reshape((denominator.shape[0],1))
  #Update of membership weights
  weights_vector= numerator/denominator

  return weights_vector

**Maximization**

In [76]:
def MaximizationStep(x,weights_vector):

  """
  Inputs: 
  x=> dataset vector whose shape is (N,M), where N correspond to the vector of
  datapoints for each modality and M corresponds to the number of modalities
  weights_vector => membership weights for all datapoints 

  output: new_parameters => dictionary containing updated parameters for the current
  iteration: alpha (mixture weights), mean and covariance.

  """ 

  #Number of datapoints for each modality
  N= x.shape[0]
  #Sum of membership weights (numerator of new mixture weights equation)
  N_k= np.sum(weights_vector, axis=0) 
  #The sum of membership weights is reshaped so that it has the dimension 
  # clusters,1
  N_k = N_k.reshape((N_k.shape[0],1))
  #New mixture weights 
  new_a= N_k/N

  #Update of the means
  #Following the formulas, dot product between weights vector and datapoints is
  #performed.
  new_mean = np.dot(np.transpose(weights_vector), x)
  new_mean = new_mean/N_k

  #Update of the covariances

  #Number of clusters
  clusters= new_mean.shape[0]
  #Number of modalities
  M = new_mean.shape[1]
  
  #New covariance matrix initialization
  new_covariance= np.zeros((clusters,M,M))

  #Calculation of new covariance matrix for each cluster based on formulas
  for c in range(clusters):
    formula_first_term=(x-new_mean[c])
    formula_second_term= (formula_first_term.transpose()*(weights_vector[:,c].reshape((1,N))))
    new_covariance[c] = np.dot(formula_second_term,formula_first_term)
    new_covariance[c] = (1/N_k[c]) * new_covariance[c]
  
  new_parameters= {'new_alpha': new_a, 'new_mean': new_mean, 'new_covariance': new_covariance}

  return new_parameters

**EM Function**

In [77]:
def EM(max_it,error_tolerance,clusters,kmeans_init, meta_data_dice, no_modalities):

  """
  Inputs: 
  max_it=> maximum numbers of iterations for the algorithm
  error_tolerance => stopping criterion for checking convergence of the algorithm
  clusters => number of tissues to be segmented
  kmeans_init => boolean variable used as a flag to select the type of parameters initialization
  if kmeans_init = True, initialization is taken from K-means algorithm. If kmeans_init = False,
  initialization is done randomly. 
  meta_data_dice => Dictionary containing feature_data (flattened image after skull stripping), brain_indices (non zero indices after excluding background),
  brain_data (brain data after excluding background ), brain_img_nifti (original image of the brain in nii format ) 

  output: final_results 

  """ 
  n_it= 1
  x = meta_data_dice['brain_data']
  #Number of voxels in the modality
  N= x.shape[0]
  #Number of modalities
  M= x.shape[1]
  #Selection of initialization:
  if kmeans_init == True:
    initial_parameters = parameters_initialization(clusters, 1, meta_data_dice, no_modalities)
    # print(initial_parameters)
  else:
    initial_parameters = parameters_initialization(clusters, 2, meta_data_dice, no_modalities)
    # print(initial_parameters)
    
  #Covariance matrix and mean are recovered from initialization of parameters
  covariance_matrix= initial_parameters['covariance']
  mean_matrix= initial_parameters['mean']

  #Mixture weights are recovered from initialization parameters
  a_vector= initial_parameters['mixture_weights']

  #Number of clusters
  clusters=mean_matrix.shape[0]

  #Generation of the matrix to store the Gaussian mixture models. This matrix 
  #will be of size NxK number of voxels in the modality x number of clusters
  gmm = np.zeros((N,clusters))
    

  #First Step: Computation of Gaussian mixture model for each tissue
  for c in range(clusters):
    covariance_matrix[c] = covariance_matrix[c] 
    gmm[:,c] = multivariate_normal.pdf(x, mean_matrix[c], covariance_matrix[c] )
    gmm[:,c] =  gmm[:,c]*a_vector[c]
  
  # After the computation of the Gaussian Mixture Model for each tissue, the 
  # initial likelihood is computed 

  c_llhd = np.log(gmm.sum(axis=1)).sum()

  # EM algorithm

  while n_it<= max_it:
    print('The algorithm is running iteration # ', n_it)

    #EXPECTATION STEP

    weights_vector= ExpectationStep(gmm)

    #MAXIMIZATION STEP

    new_parameters= MaximizationStep(x, weights_vector)

    # UPDATE OF GMM

    #Covariance matrix and mean are recovered from new parameters from the 
    #maximization step 
    covariance_matrix=  new_parameters['new_covariance']
    mean_matrix= new_parameters['new_mean']

    #Mixture weights are recovered from new parameters from 
    a_vector= new_parameters['new_alpha']

    #Number of clusters
    clusters=mean_matrix.shape[0]

    #Generation of the matrix to store the Gaussian mixture models. This matrix 
    #will be of size NxK number of voxels in the modality x number of clusters
    gmm = np.zeros((N,clusters))
    

    #Computation of Gaussian mixture model for each tissue
    for uc in range(clusters):
      covariance_matrix[uc] = covariance_matrix[uc] + np.eye(M)*1e-10
      gmm[:,uc] = multivariate_normal.pdf(x, mean_matrix[uc], covariance_matrix[uc])
      gmm[:,uc] =  gmm[:,uc]*a_vector[uc]

    #New loglikelihood

    prev_llhd= c_llhd

    c_llhd= np.log(gmm.sum(axis=1)).sum()

    #Stopping criterion: absolute differences of loglikelihood

    #Printing absolute difference of loglikelihood
    print(abs(c_llhd-prev_llhd))


    if(abs(c_llhd-prev_llhd)< error_tolerance):
      print('The algorithm has converged')
      print('The total number of iterations until convergence was:', n_it)
      final_results={'mean': mean_matrix, 'gmm': gmm,
                     'covariance': covariance_matrix, 'iterations': n_it}
      return final_results

    n_it= n_it+1
  
  print('The maximum number of iterations was reached')
  final_results={'mean': mean_matrix, 'gmm': gmm,
                     'covariance': covariance_matrix, 'iterations': n_it}
  return final_results      

### **Image Reconstruction**

In [78]:
def ImageReconstruction(em_results, meta_data_dice):

  """
  Inputs: 
  em_results => dictionary containing resulting value of the parameters ( mean, cov, gmm ) after EM
  meta_data_dice => Dictionary containing all the brain image data to be used in EM

  output: Plot Images adn Get Dice Scores.

  """ 

  brain_indices = meta_data_dice['brain_indices']
  t1_data = meta_data_dice['t1_data']
  feature_data = meta_data_dice['feature_data']
  brain_image_nifti = meta_data_dice['brain_image_nifti']
  ground_truth = meta_data_dice['ground_truth']

  seg_prob = em_results['gmm']
  centroids = em_results['mean']
  seg_labels = np.argmax(seg_prob, axis=1)

  # Sorting centroids using T1 weighted mean
  centroids_t1 = centroids[:,0]
  min_idx = np.argmin(centroids_t1, axis=0)
  max_idx = np.argmax(centroids_t1, axis=0)

  centroids_updated = np.zeros(centroids.shape)
  seg_labels_updated = np.zeros(seg_labels.shape)

  # Updating centroid after sorting according to mean values
  centroids_updated[0] = centroids[min_idx]
  centroids_updated[2] = centroids[max_idx]

  for i in [0,1,2]:
    if (min_idx != i and max_idx != i):
      mid_idx = i
      centroids_updated[1] = centroids[i]

  # Making the levels according to the ground truth (not starting from zero)
  seg_labels_updated[seg_labels==min_idx] = 1
  seg_labels_updated[seg_labels==mid_idx] = 2 
  seg_labels_updated[seg_labels==max_idx] = 3


  seg_data =  np.zeros(feature_data.shape[0])
  seg_data[brain_indices] = seg_labels_updated
  seg_result = np.reshape(seg_data, brain_image_nifti.shape)
  # Saving the images in Nifti Format
  brain_seg_data = nib.Nifti1Image(seg_result, t1_data.affine, t1_data.header)
  brain_seg_nifti = brain_seg_data.get_fdata()
  titleem = 'Image after EM'
  slice_show(brain_seg_nifti, 24, titleem)
  # Evaluation using DICE Score
  print('\nAfter applying EM:')
  dice_CSF, dice_GM, dice_WM = diceScoreSimilarity(brain_seg_nifti, ground_truth)
  print("CSF DICE = {}".format(dice_CSF), "GM DICE = {}".format(dice_GM), "WM DICE = {}".format(dice_WM))

### **Skull Stripping**

In [79]:
def skull_stripping(input_img, labeled_img):
  """
  Inputs: 
  input_img : Original given image with skull
  labeled_img : ground_truth

  output: stripped skull image.

  """ 
  labeled_img[labeled_img>0]=1
  result = np.multiply(input_img, labeled_img)
  return result

## **Workflow**

### **NIFTI Image Read and Load**

In [80]:
def loadImagedata(patient_no, modality_list):

  """
  Inputs: 
  patient_no : patient number from 1 to 5
  modality_list : A list containing the image modality names to be used

  output: 
  meta_data_dice : Dictionary containing feature_data (flattened image after skull stripping), brain_indices (non zero indices after excluding background),
  brain_data (brain data after excluding background ), brain_img_nifti (original image of the brain in nii format ) 

  """ 
  
  # Read Nifti Images
  slice_no = 24
  input_directory = '/content/drive/MyDrive/Colab Notebooks/MISA-MIRA/P2_data'
  patient = input_directory + '/' + str(patient_no)

  ground_data = nib.load(patient +'/LabelsForTesting.nii')
  ground_truth = ground_data.get_fdata()
  labeled_img = ground_truth.copy()
  # Visualize ground truth
  titleG = 'Ground Truth'
  slice_show(labeled_img, slice_no, titleG)
  flattened_list = []

  t1_data = nib.load(patient + '/'+modalities_list[0])

  for modality in modalities_list:
    data = nib.load(patient + '/'+modality)
    img = data.get_fdata()
    # Visualize the input images
    title1 = 'Original {} Image:'.format(modality)
    slice_show(img, slice_no, title1)
    brain_image = skull_stripping(img, labeled_img)
    # Saving the images in Nifti Format
    brain_image_data = nib.Nifti1Image(brain_image, data.affine, data.header)
    brain_image_nifti = brain_image_data.get_fdata()
    # Visualize the input images after skull stripping
    title2 = 'Skull-Stripped {} Image:'.format(modality)
    slice_show(brain_image_nifti, slice_no, title2)
    # Flattening and concatenating image modalities
    flattened_brain_image = brain_image_nifti.copy().flatten()
    flattened_list.append(flattened_brain_image)

  feature_data = flattened_list[0]
  for i in range(1, len(flattened_list)):
    feature_data = np.vstack((feature_data, flattened_list[i] ))
  
  feature_data = np.transpose(feature_data) 
  if(len(modality_list)==1):
    feature_data = feature_data.reshape((feature_data.shape[0],1))
    
  
  # Extracting black background from the image data considering only brain
  brain_indices = []
  for index, data in enumerate(feature_data):
    if data.any():
      brain_indices.append(index)
  # Brain Data excluding background pixels
  brain_data = feature_data[brain_indices]

  meta_data_dice = {'brain_indices': brain_indices, 't1_data': t1_data, 'feature_data': feature_data, 'brain_data': brain_data,
            'brain_image_nifti': brain_image_nifti, 'ground_truth': ground_truth}
  return meta_data_dice


### **EM**

In [ ]:
# EM Results
Patient_No = 5 # Patient Number can be from 1 to 5
modalities_list = ['T1.nii', 'T2_FLAIR.nii'] # List of Image Modalities
kmeans_init = True # Initialization by Kmeans (True) or Random(False)
meta_data_dice = loadImagedata(Patient_No, modalities_list)
max_it=500 # Number of maximum Iterations to be allowed
error_tolerance=0.001 # Tolarance of Likelihood for the convergence 
clusters = 3 # Number of tissue classes
no_modalities = len(modalities_list)
print('For Patient-{}'.format(Patient_No))
start_time = time.time()
em_results = EM(max_it, error_tolerance, clusters, kmeans_init, meta_data_dice, no_modalities) #Call EM Algorithm
ImageReconstruction(em_results, meta_data_dice) # Generate the dice scores and segmented images
end_time = time.time()
total_time = end_time - start_time
print('Total time for the computation : {}s'.format(total_time))

In [ ]:
print(np.random.normal(0, 1, size=(3,)))